# Sistema de navegación ruta - Metro CDMX
### Actividad Integradora 2 - Problema 2
**Integrantes:** Itzel Covarrubias Basurto, Karla De La Cruz Valencia

**Objetivo:** Calcular el tiempo mínimo de traslado entre estaciones considerando penalización por transbordo.

**Especificaciones:**
1. Se utiliza el algoritmo de Dijkstra.
2. Cada cambio de línea agrega 4 minutos al tiempo total.

In [48]:
import pandas as pd
import heapq
import ipywidgets as widgets
from IPython.display import display, clear_output

#Cargar datos
file_path = 'metro_cdmx_aristas.csv'
df = pd.read_csv(file_path)

#Limpieza  de espacios
df['from_stop_name'] = df['from_stop_name'].str.strip()
df['to_stop_name'] = df['to_stop_name'].str.strip()
#Convertir la línea a string 
df['route_short_name'] = df['route_short_name'].astype(str)

display(df.head())


,from_stop_id,from_stop_name,to_stop_id,to_stop_name,route_id,route_short_name,route_long_name,mean_travel_time_min
0,0200L1-BALBUENA,Balbuena,0200L1-PTOAEREO,Boulevard Puerto Aéreo,CMX0200L1,1,Pantitlán - Observatorio,1.533333
1,0200L1-BALBUENA,Balbuena,0200L1_MOCTEZUMA,Moctezuma,CMX0200L1,1,Pantitlán - Observatorio,1.633333
2,0200L1-BALDERAS,Balderas,0200L1-CUAUHTEMOC,Cuauhtémoc,CMX0200L1,1,Pantitlán - Observatorio,1.083333
3,0200L1-BALDERAS,Balderas,0200L1-SALTODELAGUA,Salto del Agua,CMX0200L1,1,Pantitlán - Observatorio,1.266667
4,0200L1-CANDELARIA,Candelaria,0200L1-MERCED,Merced,CMX0200L1,1,Pantitlán - Observatorio,1.683333


### 1. Construcción de grafo

Para manejar correctamente los transbordos definimos el grafo de la siguiente manera:
* **Nodos:** Tuplas (nombre_estacion, linea). Esto  permite diferenciar "Tacubaya Línea 1" de "Tacubaya Línea 7".
* **Aristas de viaje:** Conexiones físicas entre estaciones de la misma línea (costo = tiempo en minutos).
* **Aristas de transbordo:** Conexiones virtuales entre la misma estación de diferentes líneas (costo = 4 minutos).

In [49]:
def construir_grafo(dataframe):
    grafo = {}
    estaciones_por_nombre = {} 

    # Aristas de viaje (estación -> estación, misma línea)
    for index, row in dataframe.iterrows():
        origen = row['from_stop_name']
        destino = row['to_stop_name']
        linea = str(row['route_short_name'])
        peso = row['mean_travel_time_min']

        nodo_origen = (origen, linea)
        nodo_destino = (destino, linea)

        if nodo_origen not in grafo: grafo[nodo_origen] = []
        if nodo_destino not in grafo: grafo[nodo_destino] = []

        # Conexión bidireccional
        grafo[nodo_origen].append((nodo_destino, peso))
        grafo[nodo_destino].append((nodo_origen, peso))

        # Agrupación para detectar transbordos
        if origen not in estaciones_por_nombre: estaciones_por_nombre[origen] = set()
        estaciones_por_nombre[origen].add(linea)
        
        if destino not in estaciones_por_nombre: estaciones_por_nombre[destino] = set()
        estaciones_por_nombre[destino].add(linea)

    #Aristas de transbordo (costo de 4 minutos)
    TIEMPO_TRANSBORDO = 4.0
    
    for estacion, lineas in estaciones_por_nombre.items():
        lista_lineas = list(lineas)
        if len(lista_lineas) > 1:
            # Conectar todas las líneas de esa estación entre sí
            for i in range(len(lista_lineas)):
                for j in range(i + 1, len(lista_lineas)):
                    nodo_A = (estacion, lista_lineas[i])
                    nodo_B = (estacion, lista_lineas[j])
                    
                    grafo[nodo_A].append((nodo_B, TIEMPO_TRANSBORDO))
                    grafo[nodo_B].append((nodo_A, TIEMPO_TRANSBORDO))
                    
    return grafo, estaciones_por_nombre

grafo_metro, estaciones_dict = construir_grafo(df)
print(f"Grafo construido con {len(grafo_metro)} nodos.")

Grafo construido con 207 nodos.


### 2. Algoritmo de Dijkstra

Implementación clásica utilizando una cola de prioridad (heapq). 
El algoritmo explora el grafo buscando siempre el camino de menor costo acumulado.

In [50]:
def dijkstra_metro(grafo, estacion_inicio, estacion_fin, lineas_posibles_inicio):
    queue = []
    
    if estacion_inicio not in lineas_posibles_inicio:
        return float('inf'), []

    #Iniciar búsqueda desde todas las líneas posibles en el origen
    for linea in lineas_posibles_inicio[estacion_inicio]:
        nodo_inicial = (estacion_inicio, linea)
        # Tupla: (costo_acumulado, nodo_actual, ruta)
        heapq.heappush(queue, (0, nodo_inicial, [nodo_inicial]))
    
    visitados = set()
    costos_minimos = {} 

    while queue:
        costo_actual, nodo_actual, ruta = heapq.heappop(queue)
        nombre_est, linea_est = nodo_actual

        if nombre_est == estacion_fin:
            return costo_actual, ruta

        if nodo_actual in visitados:
            continue
        visitados.add(nodo_actual)

        if nodo_actual in grafo:
            for vecino, peso in grafo[nodo_actual]:
                nuevo_costo = costo_actual + peso
                
                if vecino not in costos_minimos or nuevo_costo < costos_minimos[vecino]:
                    costos_minimos[vecino] = nuevo_costo
                    heapq.heappush(queue, (nuevo_costo, vecino, ruta + [vecino]))
    
    return float('inf'), []

### 3. Generación de Instrucciones

Esta función toma la lista de nodos devuelta por Dijkstra y la traduce a lenguaje natural para el usuario, identificando cuándo se viaja y cuándo se transborda.

In [51]:
def obtener_peso_arista(grafo, nodo_origen, nodo_destino):
    if nodo_origen in grafo:
        for vecino, peso in grafo[nodo_origen]:
            if vecino == nodo_destino:
                return peso
    return 0

def narrativa (ruta, grafo):
    if not ruta: return ["No hay ruta disponible."]
    
    pasos = []
    tiempo_acumulado = 0.0
    
    inicio = ruta[0]
    pasos.append(f"Origen: {inicio[0]} (Línea {inicio[1]}).")
    pasos.append(f"Tiempo acumulado: 0.00 min")
    
    idx = 0
    while idx < len(ruta) - 1:
        actual = ruta[idx]
        siguiente = ruta[idx+1]
        
        tiempo_tramo = obtener_peso_arista(grafo, actual, siguiente)
        tiempo_acumulado += tiempo_tramo
        
        #CASO TRANSBORDO
        if actual[0] == siguiente[0] and actual[1] != siguiente[1]:
            # Buscar dirección
            direccion = "Desconocida"
            j = idx + 1
            while j < len(ruta) - 1:
                if ruta[j+1][1] != siguiente[1]: break
                j += 1
            direccion = ruta[j][0]

            mensaje = f"TRANSBORDO en {actual[0]}: Baje de la Línea {actual[1]} y continúe por la Línea {siguiente[1]} en dirección {direccion}."
            pasos.append(f"{mensaje}")
            pasos.append(f"   (Se agregan 4.00 min por cambio de línea). Tiempo acumulado: {tiempo_acumulado:.2f} min")

        # CASO VIAJE
        else:
            pasos.append(f"- Estación: {siguiente[0]}. Tiempo acumulado: {tiempo_acumulado:.2f} min")
            
        idx += 1
        
    pasos.append(f"\nLLEGADA: {ruta[-1][0]}.")
    pasos.append(f"Tiempo total del recorrido: {tiempo_acumulado:.2f} minutos.")
    return pasos

## 4. Interfaz de Usuario
Selecciona origen y destino para calcular la ruta.

In [52]:
lista_estaciones = sorted(list(estaciones_dict.keys()))

w_origen = widgets.Dropdown(options=lista_estaciones, description='Origen:')
w_destino = widgets.Dropdown(options=lista_estaciones, description='Destino:')
btn_calcular = widgets.Button(description='Calcular ruta')
output_area = widgets.Output()

def click(b):
    with output_area:
        clear_output()
        origen = w_origen.value
        destino = w_destino.value
        
        if origen == destino:
            print("El origen y el destino son la misma estación.")
            return
        
        print(f"\nCalculando ruta de {origen} a {destino}...")
        print("-" * 60)
        
        tiempo_total, ruta = dijkstra_metro(grafo_metro, origen, destino, estaciones_dict)
        
        if ruta:
            pasos = narrativa (ruta, grafo_metro)
            for p in pasos:
                print(p)
        else:
            print("No se encontró una ruta posible.")
        print("-" * 60)

btn_calcular.on_click(click)
display(widgets.VBox([w_origen, w_destino, btn_calcular, output_area]))